In [1]:
from hail import *
hc = HailContext()

In [5]:
vcf = '1000Genomes.ALL.coreExome10K-v1.vcf.bgz'
sample_annotations = '1000Genomes.ALL.coreExome10K-v1.sample_annotations'
pruned_variants = 'purcell5k.interval_list'

In [6]:
vds = hc.import_vcf(vcf)
vds = vds.split_multi()
vds = vds.annotate_samples_table(sample_annotations, root='sa.pheno',sample_expr='Sample', config=TextTableConfig(impute=True))
out_path = '1kg.vds'
vds.write(out_path)

In [7]:
vds = hc.read(out_path)
vds.count(genotypes=True)

{u'nGenotypes': 27786135L, u'nVariants': 10961L, u'nSamples': 2535, u'nCalled': 27417806L, u'callRate': 98.6744144156789}

In [8]:
filter_condition = '''let ab = g.ad[1] / g.ad.sum in
                          ((g.isHomRef && ab <= 0.1) || 
                           (g.isHet && ab >= 0.25 && ab <= 0.75) || 
                           (g.isHomVar && ab >= 0.9))'''
filtered_vds = vds.filter_genotypes(filter_condition)
filtered_vds.count(genotypes=True)

{u'nGenotypes': 27786135L, u'nVariants': 10961L, u'nSamples': 2535, u'nCalled': 26404807L, u'callRate': 95.02871486084696}

In [9]:
filtered_vds_2 = (filtered_vds
     .filter_variants_expr('gs.fraction(g => g.isCalled) > 0.95')
     .sample_qc())
filtered_vds_2.print_schema(sa=True)
filtered_vds_2.export_samples('sampleqc.txt', 'Sample = s.id, sa.qc.*')

In [10]:
%%sh
head sampleqc.txt | cut -f 1,2,3,4,5,6,7,8,9,10

Sample	callRate	nCalled	nNotCalled	nHomRef	nHet	nHomVar	nSNP	nInsertion	nDeletion
HG00096	9.65210e-01	5410	195	4072	682	656	1994	0	0
HG00097	9.81980e-01	5504	101	4053	835	616	2067	0	0
HG00099	9.78591e-01	5485	120	4088	770	627	2024	0	0
HG00100	9.88582e-01	5541	64	4076	902	563	2028	0	0
HG00101	9.69313e-01	5433	172	4063	744	626	1996	0	0
HG00102	9.75022e-01	5465	140	4086	753	626	2005	0	0
HG00103	9.60036e-01	5381	224	4094	615	672	1959	0	0
HG00105	9.74844e-01	5464	141	4098	765	601	1967	0	0
HG00106	9.75022e-01	5465	140	4078	778	609	1996	0	0


In [12]:
(filtered_vds_2
      .filter_samples_expr('sa.qc.callRate >= 0.97 && sa.qc.gqMean >= 20')
      .export_samples('included_samples.txt', 's.id'))
filtered_vds_3 = filtered_vds.filter_samples_list('included_samples.txt')
print 'before filter: %d samples' % filtered_vds.num_samples()
print 'after filter: %d samples' % filtered_vds_3.num_samples()
method_1_kept_ids = filtered_vds_3.sample_ids()

filtered_vds_3 = (filtered_vds
    .annotate_samples_table('sampleqc.txt', sample_expr='Sample', 
                            root='sa.qc', config=TextTableConfig(impute=True))
    .filter_samples_expr('sa.qc.callRate >= 0.97 && sa.qc.gqMean >= 20'))
print 'before filter: %d samples' % filtered_vds.num_samples()
print 'after filter: %d samples' % filtered_vds_3.num_samples()
method_2_kept_ids = filtered_vds_3.sample_ids()

method_1_kept_ids == method_2_kept_ids

post_qc_exprs = [
    'global.postQC.nCases = samples.filter(s => sa.pheno.PurpleHair).count()',
    'global.postQC.nControls = samples.filter(s => !sa.pheno.PurpleHair).count()' ]
filtered_vds_3.annotate_global_expr_by_sample(post_qc_exprs).show_globals()

filtered_vds_3 = filtered_vds_3.variant_qc()
filtered_vds_3.print_schema(va=True)
filtered_vds_3.export_variants('variantqc.tsv',
                               'Chrom=v.contig, Pos=v.start, Ref=v.ref, Alt=v.alt, va.qc.*')

before filter: 2535 samples
after filter: 1646 samples
before filter: 2535 samples
after filter: 1646 samples


In [13]:
hwe_expressions = [
    'va.hweByPop.hweEUR = gs.filter(g => sa.pheno.SuperPopulation == "EUR").hardyWeinberg()',
    'va.hweByPop.hweSAS = gs.filter(g => sa.pheno.SuperPopulation == "SAS").hardyWeinberg()',
    'va.hweByPop.hweAMR = gs.filter(g => sa.pheno.SuperPopulation == "AMR").hardyWeinberg()',
    'va.hweByPop.hweAFR = gs.filter(g => sa.pheno.SuperPopulation == "AFR").hardyWeinberg()',
    'va.hweByPop.hweEAS = gs.filter(g => sa.pheno.SuperPopulation == "EAS").hardyWeinberg()' ]
filtered_vds_3 = filtered_vds_3.annotate_variants_expr(hwe_expressions)
filtered_vds_3.persist()
filtered_vds_3.print_schema(va=True)

hwe_filter_expression = '''
    va.hweByPop.hweEUR.pHWE > 1e-6 && 
    va.hweByPop.hweSAS.pHWE > 1e-6 && 
    va.hweByPop.hweAMR.pHWE > 1e-6 && 
    va.hweByPop.hweAFR.pHWE > 1e-6 && 
    va.hweByPop.hweEAS.pHWE > 1e-6 '''
hwe_filtered_vds = filtered_vds_3.filter_variants_expr(hwe_filter_expression)
hwe_filtered_vds.count()

{u'nSamples': 1646, u'nVariants': 10135L, u'nGenotypes': 16682210L}

In [14]:
vds.filter_variants_expr('v.contig == "X"').num_variants()

273L

In [16]:
final_filtered_vds = hwe_filtered_vds.filter_variants_expr('va.qc.gqMean >= 20')
final_filtered_vds.count()

{u'nSamples': 1646, u'nVariants': 9949L, u'nGenotypes': 16376054L}

In [17]:
sex_aware_hwe_exprs = [ 
     '''va.hweByPop.hweEUR = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "EUR").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "EUR" && sa.pheno.isFemale).hardyWeinberg()''',
     '''va.hweByPop.hweSAS = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "SAS").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "SAS" && sa.pheno.isFemale).hardyWeinberg()''',
     '''va.hweByPop.hweAMR = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "AMR").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "AMR" && sa.pheno.isFemale).hardyWeinberg()''',
     '''va.hweByPop.hweAFR = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "AFR").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "AFR" && sa.pheno.isFemale).hardyWeinberg()''',
     '''va.hweByPop.hweEAS = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "EAS").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "EAS" && sa.pheno.isFemale).hardyWeinberg()''' ]
hwe_filtered_vds_fixed = (filtered_vds_3
    .annotate_variants_expr(sex_aware_hwe_exprs)
    .filter_variants_expr(hwe_filter_expression)
    .persist())
print 'total variants = %s' % hwe_filtered_vds_fixed.num_variants()
print 'X chromosome variants = %s' % hwe_filtered_vds_fixed.filter_variants_expr('v.contig == "X"').num_variants()

total variants = 10377
X chromosome variants = 268


In [18]:
sex_check_vds = (hwe_filtered_vds_fixed
    .impute_sex(maf_threshold=0.05)
    .annotate_samples_expr('sa.sexcheck = sa.pheno.isFemale == sa.imputesex.isFemale'))
total_samples = sex_check_vds.num_samples()
sex_check_passes = sex_check_vds.filter_samples_expr('sa.sexcheck').num_samples()
print 'total samples: %s' % total_samples
print 'sex_check_passes: %s' % sex_check_passes

(sex_check_vds.annotate_global_expr_by_sample(
    'global.sexcheckCounter = samples.map(s => sa.sexcheck).counter()')
    .show_globals())

sex_check_filtered_vds = sex_check_vds.filter_samples_expr('sa.sexcheck || isMissing(sa.sexcheck)').persist()
print 'samples after filter: %s' % sex_check_filtered_vds.num_samples()

total samples: 1646
sex_check_passes: 1071
samples after filter: 1643


In [19]:
pca_vds = (sex_check_filtered_vds.filter_variants_intervals('purcell5k.interval_list')
    .pca(scores='sa.pca'))
pca_vds.export_samples('pcaPlusPopulation.tsv', 
    'Sample=s, SuperPopulation=sa.pheno.SuperPopulation,'
    'Population=sa.pheno.Population, sa.pca.*')

In [22]:
rare_variant_annotations = [
    '''va.minorCase = 
        gs.filter(g => sa.pheno.PurpleHair && g.isHet).count() +
        2 * gs.filter(g => sa.pheno.PurpleHair && g.isHomVar).count()''',
    '''va.minorControl = 
        gs.filter(g => !sa.pheno.PurpleHair && g.isHet).count() + 
        2 * gs.filter(g => !sa.pheno.PurpleHair && g.isHomVar).count()''',
    '''va.majorCase = 
        gs.filter(g => sa.pheno.PurpleHair && g.isHet).count() +
        2 * gs.filter(g => sa.pheno.PurpleHair && g.isHomRef).count()''',
    '''va.majorControl = 
        gs.filter(g => !sa.pheno.PurpleHair && g.isHet).count() +
        2 * gs.filter(g => !sa.pheno.PurpleHair && g.isHomRef).count()''' ]

(sex_check_filtered_vds
    .filter_variants_expr('va.qc.AF <= 0.05 || va.qc.AF >= 0.95')
    .annotate_variants_expr(rare_variant_annotations)
    .annotate_variants_expr('''va.fet = 
                                fet(va.minorCase.toInt, va.minorControl.toInt,
                                    va.majorCase.toInt, va.majorControl.toInt)''')
    .export_variants('fisherExactTest.tsv', 'Variant = v, va.fet.*'))